In [1]:
import sys
import os
import numpy
sys.path.append('../')

# Structs

Structs are containers for structured data, similar to NumPy arrays.
The key difference to arrays is, however, that the data contained in structs is structured, i.e. hierarchical and named.
This allows writing vectorized code for much more complex data structures than multi-dimensional arrays.

The `struct` API constitues the bases for the the [`math` API](NumPy_and_TensorFlow_Execution.md).

## Examples

In [2]:
from phi import struct
a = [1, numpy.zeros([2])]
b = {'x0': 0, 'x1': 0.5, 'x2': 1}
c = (a, b)

All functions in `phi.math` can be called on structs. This broadcasts the corresponding calls to all contained arrays.

In [4]:
from phi import math
math.sin(c)

([0.8414709848078965, array([0., 0.])],
 {'x0': 0.0, 'x1': 0.479425538604203, 'x2': 0.8414709848078965})

Note that math.sin calls numpy.sin in this case.
If the struct contained TensorFlow tensors, tensorflow.sin would be called instead.

## Types of structs

The following types are considered structs:
- Lists
- Tuples
- Dicts containing strings as keys
- NumPy arrays with `dtype=numpy.object`
- Subclasses of [`phi.math.struct.Struct`](../phi/struct/struct.py)

All `phi.math` functions and functions of the struct API work with any of the above types.

While all entries of lists, tuples, dicts and NumPy arrays are expected to hold data,
subclasses of the `Struct` class can define further properties which are not subject to the above mentioned functions.

Struct items are categorized into *variables* and *constants*. Variables are expected to change when a struct is run through an algorithm while constants should generally only be changed by the user. Typically, variables hold data in the form of tensors or grids while constants hold scalar values, booleans or strings.

## Iterating over structs

The struct interface provides the function `map` which iterates over all data-holding items of a struct and its sub-structs by default.

In [5]:
struct.map(lambda x: str(x), c)

(['1', '[0. 0.]'], {'x0': '0', 'x1': '0.5', 'x2': '1'})

If we only wanted to affect the tensors directly held by `data`, we could call

In [6]:
struct.map(lambda x: str(x), c, recursive=False)

('[1, array([0., 0.])]', "{'x0': 0, 'x1': 0.5, 'x2': 1}")

Assume, we want only dicts to be stringified as a whole.
This can be achieved by defining a leaf_condition.

In [7]:
struct.map(lambda x: str(x), c, leaf_condition=lambda x: isinstance(x, dict))

(['1', '[0. 0.]'], "{'x0': 0, 'x1': 0.5, 'x2': 1}")

The parameter `item_condition` can further specify which types of items should be affected by a struct operation.
The constants `VARIABLES`, `CONSTANTS`, `DATA` and `ALL_ITEMS` are part of the `struct` package and can be used for the `item_condition`.

In some cases we require additional information when mapping a struct; not just the value but also where it is stored.
When calling `map(.., trace=True)`, a [`Trace`](../phi/struct/functions.py) object is passed to the mapping function `f` instead of the value. In addition to retrieving the value via `trace.value`, it provides access to the attribute key as `trace.key` and the parent structs via `trace.parent`.

In [8]:
struct.map(lambda trace: trace.key, c, trace=True)

([0, 1], {'x0': 'x0', 'x1': 'x1', 'x2': 'x2'})

## Usages in Φ<sub>*Flow*</sub>

In Φ<sub>*Flow*</sub>, structs are mostly used to store simulation states, i.e.
each attribute holds a tensor such as density or velocity of a fluid simulation.
In particular, the state base class [`phi.physics.physics.State`](../phi/physics/physics.py) extends `Struct`.
All Field classes such as StaggeredGrid are also structs.

Properties are used to hold additional parameters for the simulation that should be included in the `description.json` file. Typical examples of these include `viscosity` or `buoyancy_factor`.

In [10]:
from phi.flow import *
fluid = Fluid(Domain([80, 64]))

def print_name(trace):
    print('%s  (%s)' % (trace.path(), type(trace.value).__name__))
    return trace.value
struct.map(print_name, fluid, trace=True, item_condition=None);

buoyancy_factor  (float)
domain.boundaries.friction  (float)
domain.boundaries.name  (str)
domain.boundaries.periodic  (bool)
domain.boundaries.solid  (bool)
domain.resolution  (ndarray)
domain.box.lower  (ndarray)
domain.box.upper  (ndarray)
density.data  (ndarray)
density.box.lower  (ndarray)
density.box.upper  (ndarray)
density.extrapolation  (str)
density.interpolation  (str)
density.age  (float)
density.name  (str)
density.tags.0  (str)
density.tags.1  (str)
velocity.resolution  (ndarray)
velocity.box.lower  (ndarray)
velocity.box.upper  (ndarray)
velocity.data.0.data  (ndarray)
velocity.data.0.box.lower  (ndarray)
velocity.data.0.box.upper  (ndarray)
velocity.data.0.extrapolation  (str)
velocity.data.0.interpolation  (str)
velocity.data.0.age  (float)
velocity.data.0.name  (str)
velocity.data.0.tags.0  (str)
velocity.data.0.tags.1  (str)
velocity.data.1.data  (ndarray)
velocity.data.1.box.lower  (ndarray)
velocity.data.1.box.upper  (ndarray)
velocity.data.1.extrapolation  (str)
v

[Staggered grids](Staggered_Grids.md), as in `fluid.velocity`, are vector fields where the arrays for each component have different shapes.

## Tensor initialization

Initializer functions such as `zeros` or `placeholder` internally call their counterparts in NumPy or TensorFlow.
They can take 1D-tensors describing the shape as input but also support structs holding shapes.
The call `zeros(StaggeredGrid([1,65,65,2]))` will return a `StaggeredGrid` holding a NumPy array.

Some states simplify this even further by allowing a syntax like `Fluid(density=zeros)` or `Fluid(velocity=placeholder)`.

The `placeholder` and `variable` initializers also infer the name of the resulting tensors from the attribute names.

### Data I/O

The data writing and reading system accepts structs and automatically infers their names from the attributes.
See the [data documentation](Reading_and_Writing_Data.md).

### Session

The [`Session`](../phi/tf/session.py) class is a customized version of `tf.Session` which accepts structs for the `fetches` argument as well as inside the `feed_dict`.

This can be used to quickly run states through a graph like so:

In [13]:
from phi.tf.flow import *
numpy_state = Fluid(Domain([16, 16]), density=math.zeros, velocity=math.zeros)
placeholder_state = Fluid(Domain([16, 16]), density=placeholder, velocity=placeholder)
output_state = INCOMPRESSIBLE_FLOW.step(placeholder_state)
session = Session(None)
numpy_state = session.run(output_state, {placeholder_state: numpy_state})
numpy_state

Fluid[density: Grid[16x16(1), size=[16. 16.]], velocity: StaggeredGrid[16x16, size=[16. 16.]]]

## Validity

As structs are supposed to hold data in a specific structure, there is a preferred data type for each entry.
For a CenteredGrid, the `data` attribute should be a tensor or array with a certain rank and the `velocity` of a `Fluid` object should be a `StaggeredGrid`.

An entry is _valid_ if its value if of the preferred data type.
Subclasses of `Struct` can implement validity checks and modify their entries to make them valid.

This hierarchy is not always needed, however. Many math functions return invalid structs such as `math.staticshape(obj)` which returns a struct containing shapes instead of data.
Code dealing with invalid structs should always be enclosed in a `with struct.unsafe():` block.
This context skips all data validation steps.

# Immutability
While structs can be mutable in principle, the struct interface does not allow for changing a struct.
Attributes and properties can be "changed" using the `copy_with` function.
In this way, the struct isn't altered but rather a duplicate with the new values is created.

## Implementing a custom struct

Variables and constants are declared using the decorators `struct.variable()` and `struct.constant()`, respectively.
Variables are marked as data-holding by default, constants as non-data.
These items create read-only attributes which should be changed only using the inherited `copied_with()` method.

The `generate` method can be used to quickly generate the Python skeleton of a custom struct.

In [16]:
from phi.struct.python_generator import generate
print(generate('MyStruct', attributes=['a'], properties=['p'], other=['other']))


from phi import struct


@struct.definition()
class MyStruct(struct.Struct):
    
    def __init__(self, a, p, other, **kwargs):
        struct.Struct.__init__(self, **struct.kwargs(locals(), ignore=['other']))
        self._other = other
    
    @struct.variable()
    def a(self, a):
        return a

    @struct.constant()
    def p(self, p):
        return p
        
    @property
    def other(self):
        return self._other
        



The methods themselves are used for validation. In addition to `self`, each attribute and property gets the intended value as an input. The function can either directly return this value without any validity checks, raise an error for invalid values or transform the value into a valid value.

Unless created inside a `with struct.unsafe()` block, structs are always valid when viewed from outside.

In [20]:
from phi import struct


@struct.definition()
class MyStruct(struct.Struct):
    
    def __init__(self, a, p, other, **kwargs):
        struct.Struct.__init__(self, **struct.kwargs(locals(), ignore=['other']))
        self._other = other
    
    @struct.variable()
    def a(self, a):
        return a

    @struct.constant()
    def p(self, p):
        return str(p)
        
    @property
    def other(self):
        return self._other

mystruct = MyStruct(a=0, p=0, other=None)
print(mystruct.p, type(mystruct.p))
mystruct = mystruct.copied_with(p=1)
print(mystruct.p, type(mystruct.p))

0 <class 'str'>
1 <class 'str'>
2 <class 'int'>
